<a href="https://colab.research.google.com/github/monocleface/huggingface-tutorials/blob/main/sample_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text Classification(텍스트 분류)은 주어진 텍스트(문장이나 문서)를 미리 정해진 카테고리(범주) 중 하나로 분류하는 작업을 의미합니다.

가장 대표적인 예시가 바로 감성 분석(Sentiment Analysis)입니다.

# 텍스트 분류란?
텍스트를 읽고, 그 텍스트가 어떤 종류에 속하는지 꼬리표(Label)를 붙이는 모든 작업을 말합니다.
예를 들어, "이 영화 정말 재밌어요!"라는 텍스트가 있다면, 텍스트 분류 모델은 이 문장을 "긍정"이라는 꼬리표로 분류합니다.

# 주요 용도 및 예시
텍스트 분류는 매우 다양한 곳에 사용됩니다.

## 감성 분석 (Sentiment Analysis):
- 목적: 문장이 긍정적인지, 부정적인지, 중립적인지 판단합니다.
- 예시: 영화 리뷰, 쇼핑몰 상품평, 소셜 미디어 반응 분석

예를 들어 "배송도 빠르고 제품도 마음에 듭니다."라는 문장은 -> (출력) 긍정(Positive)으로 분류됩니다.

## 뉴스 기사 분류 (News Categorization):
- 목적: 기사 내용을 바탕으로 카테고리를 분류합니다.
- 예시: 네이버나 다음 뉴스 섹션

예를 들어 "A 선수가 해트트릭을 기록하며 팀을 승리로 이끌었습니다..."라는 문장은 -> (출력) 스포츠(Sports)로 분류됩니다.

## 스팸 메일 필터링 (Spam Detection):
- 목적: 이메일 내용이 정상 메일인지 스팸(광고) 메일인지 구분합니다.
- 예시: Gmail, 네이버 메일의 스팸함

예를 들어 "(광고) 특별 할인! 지금 바로 확인하세요..."라는 문장은 -> (출력) 스팸(Spam)으로 분류됩니다.

## 주제 분류 (Topic Classification):
- 목적: 고객센터 문의 내용이나 게시판 글의 주제를 분류합니다.
- 예시: "로그인이 안 돼요" → 계정 문의, "결제 취소해 주세요" → 결제 문의

Huggingface에 있는 모델을 가지고 위에선 언급된 항목별로 샘플 테스트를 합니다.

샘플 코드를 살펴보시죠.

In [1]:
# huggingface transformers 설치
!pip install transformers